In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import LSTM, Dense

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Dense, Flatten
from tensorflow.keras.layers import BatchNormalization


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve

tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')


In [2]:
df = pd.read_pickle('prepped_train.pkl')
df_labels = pd.read_csv('labels_q.csv')

df_labels = df_labels[['session_id', 'q2']]
df_labels.columns = ['session_id', 'label']

df = df.merge(df_labels, on='session_id')

In [3]:
def create_sequence(session_id):
    sequence = df[df['session_id'] == session_id]

    X = sequence.drop(['session_id','label'], axis=1).values
    y = sequence['label'].iloc[-1]

    # if sequence is not 220, pad with zeros
    if X.shape[0] < 220:
        X = np.pad(X, ((0, 220-X.shape[0]), (0,0)), 'constant')

    if X.shape[0] > 220:
        X = X[:220]
    return X, y

Xs, ys = [], []

for session_id in df['session_id'].unique():
    X, y = create_sequence(session_id)
    if X.shape[0] > 1: # filter out sessions with only one event
        Xs.append(X)
        ys.append(y)

Xs = np.array(Xs)
ys = np.array(ys)

X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.2, random_state=42)

# get validation set which is half of the test set
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [4]:
learning_rate = 0.0005
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)

model = Sequential()
model.add(BatchNormalization(input_shape=(220, X_train.shape[2])))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss=tf.keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True), optimizer=optimizer, metrics=['accuracy'],)

#model.compile(loss=tf.keras.losses.BinaryFocalCrossentropy(), optimizer='adam', metrics=['accuracy'],)

#model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=8, batch_size=32)

y_pred = model.predict(X_test)


threholds = np.arange(0.0, 1.00, 0.01)
f1_scores = []

for threhold in threholds:
    f1_scores.append(f1_score(y_test, y_pred > threhold, average='macro'))

best_threshold = threholds[np.argmax(f1_scores)]


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/8


2023-04-13 22:02:38.151761: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


590/590 [==============================] - 29s 45ms/step - loss: 0.0172 - accuracy: 0.9751 - val_loss: 0.0163 - val_accuracy: 0.9788
Epoch 2/8
590/590 [==============================] - 26s 44ms/step - loss: 0.0164 - accuracy: 0.9784 - val_loss: 0.0163 - val_accuracy: 0.9788
Epoch 3/8
590/590 [==============================] - 26s 43ms/step - loss: 0.0164 - accuracy: 0.9786 - val_loss: 0.0161 - val_accuracy: 0.9788
Epoch 4/8
590/590 [==============================] - 26s 44ms/step - loss: 0.0164 - accuracy: 0.9786 - val_loss: 0.0164 - val_accuracy: 0.9788
Epoch 5/8
590/590 [==============================] - 26s 44ms/step - loss: 0.0162 - accuracy: 0.9787 - val_loss: 0.0161 - val_accuracy: 0.9771
Epoch 6/8
590/590 [==============================] - 26s 43ms/step - loss: 0.0162 - accuracy: 0.9785 - val_loss: 0.0158 - val_accuracy: 0.9784
Epoch 7/8
590/590 [==============================] - 26s 44ms/step - loss: 0.0159 - accuracy: 0.9786 - val_loss: 0.0166 - val_accuracy: 0.9779
Epoch 8/8

In [5]:
y_pred = model.predict(X_val)

f1_score(y_val, y_pred > best_threshold, average='macro')

74/74 [==============================] - 1s 14ms/step


0.5079316137196522